# Практическое задание: Визуализация и применение оптического потока
В данном задании Вы попрактикуетесь в применении методов вычисления оптического потока и его визуализации.

## Часть 0: Подготовка
Для работы с методами оптического потока и удобной визузализации нам будут необходимы следующие библиотеки:
- opencv
- pyflow (https://github.com/pathak22/pyflow)
- imageio
- matplotlib
- scikit-image
- PIL

Убедитесь, что они у вас установлены.

In [ ]:
import numpy as np
from skimage.io import imread, imshow, imsave
import IPython
import cv2
from pyflow import pyflow
import matplotlib.pyplot as plt
from math import gcd
import imageio
from PIL import Image, ImageSequence
import warnings


def gifread(gifname):
    im = Image.open(gifname)
    duration = im.info['duration']/1000.
    frames = [(np.array(frame.copy().convert('RGB'))/255.).astype(np.float64).copy()
              for frame in ImageSequence.Iterator(im)]
    return frames, duration


def gifshow(gifname):
    with open(gifname, 'rb') as f:
        display(IPython.display.Image(data=f.read(), format='png'))


def gifsave(gifname, images, duration=2):
    images = [imageio.core.util.Image(img) for img in images]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        imageio.mimsave(gifname, images, duration=duration)


def seqmerge(imgs1, imgs2, mode='horizontal'):
    len_gcd = gcd(len(imgs1), len(imgs2))
    imgs1_repeat = len(imgs2)//len_gcd
    imgs2_repeat = len(imgs1)//len_gcd
    imgs1 = np.repeat(imgs1, imgs1_repeat, axis=0)
    imgs2 = np.repeat(imgs2, imgs2_repeat, axis=0)
    imgs = np.concatenate(
        (imgs1, imgs2), axis=2 if mode.startswith('hor') else 1)
    return imgs

Визуализировать результат будет удобно в виде GIF-анимаций, для чего вы можете использовать функции `gifread`, `gifsave`, `gifshow` и `seqmerge`, пример использования ниже.

In [ ]:
frames, duration = gifread('data/cat.gif')
flipped_frames = [f[::-1, ...] for f in frames[::-1]]
res_frames = seqmerge(frames, flipped_frames, mode='vertical')
gifsave('double_cat.gif', res_frames, duration=duration)
gifshow('double_cat.gif')

## Часть 1: Что такое оптический поток

Оптический поток – это мера видимого движения объектов на видеопоследовательности относительно плоскости изображения. На практике оптический поток разделяется на 2 типа: __плотный__ (dense) и __разреженный__ (sparse). С помощью плотного оптического потока производится оценка смещения каждого пикселя изображения, в том время как с помощью разреженного оценивают только некоторые, заранее определенные точки, что значительно экономит ресурсы по сравнению с первым методом.

## Часть 2: Плотный оптический поток и его визуализация
<img src="imgs/dof.jpg" width=350 align="right">
Для расчета плотного оптического потока применим быстрый метод расчета по пирамиде изображений (https://people.csail.mit.edu/celiu/OpticalFlow/). Для визуализации полученного оптического потока можно применить следующий метод: направлению вектора будет соответствовать Hue в цветовой модели HSV, а абсолютному значению – Value

__Задание 1.__ Сделайте визуализацию оптического потока по HSV. Подберите оптимальные параметры для вычисления оптического потока между двумя кадрами в коде ниже.

In [ ]:
im1 = np.array(Image.open('data/car1.jpg'))
im2 = np.array(Image.open('data/car2.jpg'))
im1 = im1.astype(float) / 255.
im2 = im2.astype(float) / 255.

# Parameters to tune
alpha = 0.012
ratio = 0.75
minWidth = 20
nOuterFPIterations = 7
nInnerFPIterations = 1
nSORIterations = 30
colType = 0

u, v, im2W = pyflow.coarse2fine_flow(
    im1, im2, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
    nSORIterations, colType)
flow = np.concatenate((u[..., None], v[..., None]), axis=2)

# Your code here

imshow(rgb)

Плотный оптический поток имеет несколько применений. Одно из них – интерполирование видеопоследовательностей для увеличения частоты кадров и плавности движения. С помощью вычисленного на предыдущем шаге потоке попробуйте сгенерировать кадр между двумя настоящими.

__Задание 2.__ Реализуйте функцию `warp_flow`, выполняющую интерполяцию кадров по потоку и первому изображению. *Подсказка:* Используйте функцию `cv2.remap`, для неё пиксели должны быть в формате `np.float32`. Не забудьте в конце привести значения пикселей к [0,1].

In [ ]:
def warp_flow(img, flow):
    # Your code here

In [ ]:
im3 = warp_flow(im1, flow.astype(np.float32).copy())
seq = seqmerge([im1, im1, im2], [im1, im3, im2])
gifsave('warped.gif', seq, duration=1.5)
gifshow('warped.gif')

Попробуем применить данный метод для видео. Заметно ли увеличение плавности?

__Задание 3.__ Реализуйте функцию `interp_frames`, вычисляющую плотный оптический поток между соседними кадрами и интерполирующую кадр между ними. Примените функцию к gifs/traffic.gif и визуализируйте результат в виде анимации, сравните с оригиналом. *Подсказка:* Не забудьте уменьшить параметр `duration` для итоговой анимации в 2 раза.

In [ ]:
def interp_frames(frames):
    # Your code here

In [ ]:
# Visualise here

## Часть 3. Разреженный оптический поток
<img src="imgs/opticalflow_lk.jpg" width=350 align="right">

Методы разреженного оптического потока чаще всего работают медленней, чем плотного, из расчета на одну отслеживаемую точку, однако отличаются большей надежностью отслеживания необходимых точек и потому применяются в задачах трекинга объектов.

Попробуем рассчитать поток по методу Лукаса-Канаде (https://en.wikipedia.org/wiki/Lucas–Kanade_method). Для этого сначала необходимо задать интересующие нас точки, найдем их с помощью `cv2.goodFeaturesToTrack`. Затем применим метод к каждой паре соседних кадров и получим на выходе положения интересующих точек на каждом кадре последовательности.

__Задание 4.__ Напишите функцию визуализации треков получаемых точек `sparseflow_visualise`. На каждом кадре функция должна продолжать линии разных цветов согласно точкам, чтобы получилось изображение, похожее на пример справа. Примените функцию вычисления разреженного оптического потока `compute_sparse_flow` к gifs/traffic.gif. Визуализируйте результат в виде анимации. Подберите оптимальные параметры для предотвращения потери ключевых точек.

In [ ]:
def compute_sparse_flow(images):
    def to_uint8(images):
        if images[0].dtype != np.uint8:
            images = [(img*255).astype(np.uint8).copy() for img in images]
        return images
    res = []
    images = to_uint8(images)
    # params for ShiTomasi corner detection
    feature_params = dict(maxCorners=30,
                          qualityLevel=0.4,
                          minDistance=7,
                          blockSize=7)
    # Parameters for Lucas-Kanade optical flow
    lk_params = dict(winSize=(7, 5),
                     maxLevel=40,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 40, 0.1))

    prev_gray = cv2.cvtColor(images[0], cv2.COLOR_RGB2GRAY)
    prev_features = cv2.goodFeaturesToTrack(
        prev_gray, mask=None, **feature_params).reshape(-1, 2)
    res.append(prev_features)
    features_len = len(prev_features)
    feature_mask = np.ones(features_len, dtype=np.bool)
    for image in images[1:]:
        curr_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        curr_features, curr_feature_mask, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray,
                                          prev_features.reshape(-1, 1, 2).copy(), None, **lk_params)
        curr_feature_mask = curr_feature_mask.flatten()
        prev_features = curr_features[curr_feature_mask == 1, ...].reshape(-1, 2).copy()
        feature_mask[feature_mask] = (curr_feature_mask == 1)
        features_to_add = np.ones((features_len, 2)) * np.inf
        features_to_add[feature_mask, :] = prev_features
        res.append(features_to_add)

    return res

In [ ]:
def sparseflow_visualise(images, points):
    # Your code here

In [ ]:
# Visualise here

Вы можете заметить, как трекинг практически всегда срывается при значительном изменении окружения точки (в данном случае, изменение освещения корпусов машин, когда они выезжают на солнце). Можно сделать вывод, что для качественного вычисления оптического потока необходимо более глубокое понимание сцены, чем простое сравнение областей, именно поэтому в данный момент наиболее перспективными методами решения данной задачи являются методы с применением нейросетей. Один из самых известных примеров – FlowNet, подробней о котором можно прочесть здесь: https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Dosovitskiy_FlowNet_Learning_Optical_ICCV_2015_paper.pdf